# Claim detection
The dataset(IBM Debator - Claim Stance Dataset) can be accessed [here](http://www.research.ibm.com/haifa/dept/vst/debating_data.shtml)

This dataset contains: 2,394 labeled claims for 55 topics. The dataset includes the stance (Pro/Con) of each claim towards the topic, as well as fine-grained annotations, based on the semantic model of Bar-Haim et al. [2017a] (topic target, topic sentiment towards its target, claim target, claim sentiment towards its target, and the relation 
between the targets).

The dataset includes: 
1. A utf-8 JSON file containing the topics and the claims found for these topics in Wikipedia articles. 
   Topics and claims are annotated as described above. 
2. A utf-8 CSV file containing the same information as the JSON file.
3. The original Wikipedia articles - from Wikipedia April 2012 dump - in the form of text files. 
   For each article, we provide both the original (raw) version, and a clean version, in which any Wikisyntax 
   and HTML markup is removed.
4. A CSV index file, containing the article title and Wikipedia URL for each article.   


In [2]:
import numpy as np
import pandas as pd
import string
import nltk

In [3]:
import os
root = os.getcwd() + '/IBM_EACL-2017.v1/'

# 1. Reading in data

### articles
num: 547

Note: There're 5013 texts within articles folder, so this dataframe only contains articles that are labeled as one of the 58 topics

In [5]:
def read_file(filename):
    data = []
    with open(filename, 'r') as f:
        text = f.read().split('\n')
        for parag in text:
            if len(parag) >2:
                data = data + [a for a in parag.split('. ') if len(a)>0]
                
    return data

In [6]:
articles = dict()
for subdir, dirs, files in os.walk(root+'articles/'):
    for file in files:
        if file[-3:] == 'txt' and file[:5] == 'clean':
            filename = os.path.join(subdir, file)
            articles[file] = read_file(filename)


In [8]:
articles['clean_1.txt'][:2]

['Controversies over video games often center on topics such as video game graphic violence, sex and sexism, violent and gory scenes, partial or full nudity, portrayal of criminal behavior, racism, and other provocative and objectionable material',
 'Video games have been studied for links to addiction and aggression']

In [61]:
article_paths = set(claimDF['claims.article.cleanFile'])

### claims
num: 2394

In [9]:
claimDF = pd.read_csv("IBM_EACL-2017.v1/claim_stance_dataset_v1.csv")

In [10]:
claimDF[:1].T

,0
topicId,1
split,test
topicText,This house believes that the sale of violent v...
topicTarget,the sale of violent video games to minors
topicSentiment,-1
claims.claimId,2973
claims.stance,PRO
claims.claimCorrectedText,Exposure to violent video games causes at leas...
claims.claimOriginalText,exposure to violent video games causes at leas...
claims.article.rawFile,articles/t1/raw_1.txt


In [ ]:
# I've just realized that one sentence could contain more than one claims, 
# so I'll just tag it as 1 as long as it contains one claim

In [186]:
articles_tags = []
articles_sent_length = []
sentences = []
topics = []
article_locs = []

for a in article_paths:
    article_name = a.split('/')[-1]
    sents = articles[article_name]
    claims = list(claimDF[claimDF['claims.article.cleanFile'] == a]['claims.claimOriginalText'])
    topic = list(claimDF[claimDF['claims.article.cleanFile'] == a]['topicTarget'])[0]
    tags = []
    sent_length = []
    locs = []

    for i, sent in enumerate(sents):
        locs.append(i)
        # get sent length
        tokens = [a for a in sent.split() if a != '']
        sent_length.append(len(tokens))
        topics.append(topic)
        flag = False
        for claim in claims:
            if claim in sent:
                tags.append(1)
                claims.remove(claim)
                flag = True
                break
        if not flag:
            tags.append(0)  
    
    sentences.append(sents)
    articles_tags.append(tags)
    article_locs.append(locs)
    articles_sent_length.append(sent_length)

In [187]:
d = {'sentences': sum(sentences, []), 
     'topic': topics, 
     'length': sum(articles_sent_length, []), 
     'label': sum(articles_tags, []),
    'loc': sum(article_locs, [])}
tmp = pd.DataFrame(d)

In [188]:
data = list(zip(sum(sentences, []), topics,\
           sum(articles_sent_length, []), sum(articles_tags, []), sum(article_locs, [])))

# sentence, topic, length, label, loc

In [189]:
data[0]

('Brown v', 'the sale of violent video games to minors', 2, 0, 0)

# 2.  Feature Engineeing

- Mean embedding of the sentence
- similarity between sentence and topic (cosine similarity )
- length of the sentence
- index of the sentence in the article


## Ignore codes starting from here
## Organizing data into ready-to-train format

functions

In [13]:
def clean(x):
    return x.lower().strip(' ').translate(str.maketrans('','',string.punctuation))

In [14]:
def read_text(x):
    t = ''
    filename = "CE-EMNLP-2015.V3/articles/clean_"+str(x)+".txt"
    for line in open(filename, 'r'):
        t += line
    return t

def find_claim(x, dic):
    # finding claims from dictionary{article id: [claims]}
    try:
        return dic[x]
    except:
        return []

clean up claimDF

In [15]:
claimDF['original_rmv_punc'] = claimDF['Claim original text'].apply(lambda x: clean(x))
#claimDF['Claim corrected version'] = claimDF['Claim corrected version'].apply(lambda x: clean(x))

In [183]:
claimDF[:2]

,topicId,split,topicText,topicTarget,topicSentiment,claims.claimId,claims.stance,claims.claimCorrectedText,claims.claimOriginalText,claims.article.rawFile,...,claims.article.rawSpan.end,claims.article.cleanFile,claims.article.cleanSpan.start,claims.article.cleanSpan.end,claims.Compatible,claims.claimTarget.text,claims.claimTarget.span.start,claims.claimTarget.span.end,claims.claimSentiment,claims.targetsRelation
0,1,test,This house believes that the sale of violent v...,the sale of violent video games to minors,-1,2973,PRO,Exposure to violent video games causes at leas...,exposure to violent video games causes at leas...,articles/t1/raw_1.txt,...,640,articles/t1/clean_1.txt,418,568,yes,Exposure to violent video games,0.0,31.0,-1.0,1.0
1,1,test,This house believes that the sale of violent v...,the sale of violent video games to minors,-1,2974,CON,video game violence is not related to serious ...,video game violence is not related to serious ...,articles/t1/raw_1.txt,...,1697,articles/t1/clean_1.txt,829,907,yes,video game violence,0.0,19.0,1.0,1.0


Get original text of these articles

In [17]:
artDF['text'] = artDF['article Id'].apply(lambda x: read_text(x))

In [18]:
artDF[:4]

,Topic,Title,article Id,text
0,This house believes that the sale of violent v...,Video game content rating system,567,A video game content rating system is a system...
1,This house believes that the sale of violent v...,Grand Theft Childhood,524,Grand Theft Childhood: The Surprising Truth Ab...
2,This house believes that the sale of violent v...,School violence,542,School violence is widely held to have become ...
3,This house believes that the sale of violent v...,Video game controversies,1,. \n\nControversies over video games often cen...


For each article, find the extracted claimes

In [19]:
#claimDF.iloc[0,:]

Topic                      This house believes that the sale of violent v...
Claim original text        Exposure to violent video games causes at leas...
Claim corrected version    exposure to violent video games causes at leas...
original_rmv_punc          exposure to violent video games causes at leas...
Name: 0, dtype: object

In [20]:
dic = {}
notFound = []
for i in range(len(claimDF)):
    topic = claimDF.iloc[i,0]
    claim = str(claimDF.iloc[i,3])
    df = artDF[artDF['Topic']==topic]
    flag = False
    for j in range(len(df)):
        if claim in df.iloc[j,3]:
            flag = True
            try:
                dic[df.iloc[j,2]].append(claim)
            except:
                dic[df.iloc[j,2]] = [claim]
    if flag == False:
        notFound.append((i, topic, claim))

In [21]:
artDF['claims'] = artDF['article Id'].apply(lambda x: find_claim(x,dic))

artDF['num_claim'] = artDF['claims'].apply(lambda x: len(x))

artDF['num_claim'].sum()

726

In [22]:
len(notFound)

1634

In [24]:
artDF['sentences'] = artDF['text'].apply(lambda x: [clean(sent) for sent in x.split('.')])
artDF['num_sents'] = artDF['sentences'].apply(lambda x: len(x))

In [25]:
artDF[:4]

,Topic,Title,article Id,text,claims,num_claim,sentences,num_sents
0,This house believes that the sale of violent v...,Video game content rating system,567,A video game content rating system is a system...,[numerous researchers have proposed potential ...,1,[a video game content rating system is a syste...,107
1,This house believes that the sale of violent v...,Grand Theft Childhood,524,Grand Theft Childhood: The Surprising Truth Ab...,[most children who play violent games do not h...,1,[grand theft childhood the surprising truth ab...,60
2,This house believes that the sale of violent v...,School violence,542,School violence is widely held to have become ...,[],0,[school violence is widely held to have become...,136
3,This house believes that the sale of violent v...,Video game controversies,1,. \n\nControversies over video games often cen...,[video game violence is not related to serious...,10,"[, \n\ncontroversies over video games often ce...",299


In [26]:
#artDF['claims'][0]

In [27]:
#artDF['sentences'][0]

In [28]:
# def determine_label(x):
#      if x

SyntaxError: invalid syntax (<ipython-input-28-06db512e1137>, line 2)

In [29]:
labels = []
for i in range(len(artDF)):
    sents = artDF['sentences'][i]
    claims = artDF['claims'][i]
    label = []
    for sent in sents:
        flag = False
        for claim in claims:
            if claim in clean(sent):
                label.append(1)
                flag = True
                break
        if flag == False:
            label.append(0)
    labels.append(label)

In [30]:
artDF['label'] = labels

In [31]:
sum(artDF['label'][0])

1

In [32]:
artDF['sum_label'] = artDF['label'].apply(lambda x: sum(x))

In [33]:
sum(artDF['sum_label'])

670

Problem: I keep losing claims!!

## What if I don't keep track of the article that the sentences are in, and just determine them altogether.

In [34]:
all_text = ''
for i in artDF['text']:
    all_text += i

In [35]:
all_sents = all_text.split('. ')

In [41]:
claims = claimDF['original_rmv_punc']
labels = []
for sent in all_sents:
    sent = clean(sent)
    flag = False
    for claim in claims:
        if claim in sent:
            labels.append(1)
            flag = True
            break
    if flag == False:
        labels.append(0)

In [37]:
len(all_sents)

75675

In [42]:
len(labels)

75675

In [43]:
sum(labels)

1702

In [44]:
len(claims)

2294

Major problem: when there suppose to be 2294 claims, I'm only able to find 1702 sentences that contain claims using this method.

In [46]:
d = {'Sent': all_sents, 'Contains_claims': labels}
sentDF = pd.DataFrame(d)

In [61]:
sentDF[:5]

,Contains_claims,Sent,?
0,0,A video game content rating system is a system...,0
1,0,Most of these systems are associated with and/...,0
2,0,The utility of such ratings has been called in...,0
3,0,Video game content rating systems can be used ...,0
4,0,Rating checking and approval is part of the ga...,0


# Feature extraction
1. ？

In [58]:
def qMark(x):
    try:
        if x[-1] == '?':
            return 1
        else:
            return 0
    except:
        return 0

In [59]:
sentDF['?'] = sentDF['Sent'].apply(lambda x: qMark(x))

2. Belief words

# Text rank/text summarization

In [82]:
import sumy
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
#from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
#from sumy.utils import get_stop_words

In [70]:
t = artDF['text'][0]

In [90]:
artDF['claims'][0]

['numerous researchers have proposed potential positive effects of video games']

In [84]:
SENTENCES_COUNT = artDF['num_sents'][0]

In [86]:
LANGUAGE = 'English'
parser = PlaintextParser.from_string(t, Tokenizer('English'))
stemmer = Stemmer(LANGUAGE)

summarizer = TextRankSummarizer(stemmer)
summarizer.stop_words = nltk.corpus.stopwords.words('english')
# summarizer = Summarizer(stemmer)
# summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

TypeError: 'NoneType' object is not iterable

## raw textrank
https://joshbohde.com/blog/document-summarization

In [87]:
import networkx as nx
import numpy as np
 
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
 
def textrank(document):
    sentence_tokenizer = PunktSentenceTokenizer()
    sentences = sentence_tokenizer.tokenize(document)
 
    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)
 
    similarity_graph = normalized * normalized.T
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i],s) for i,s in enumerate(sentences)),
                  reverse=True)

In [91]:
textrank(t)[:10]

[(0.018228815830554752,
  'The (CERO) is the organization that rates video games and PC games (except dating sim, visual novel, and eroge) in Japan with levels of rating that informs the customer of the nature of the product and for what age group it is suitable.'),
 (0.015257808196775656,
  'It is responsible for inspecting and rating the content of films and interactive games.'),
 (0.014390936223200246,
  'A video game content rating system is a system used for the classification of video games into suitability-related groups.'),
 (0.014305087014280488,
  'In July 1994, the IDSA (renamed to the Entertainment Software Association in 2003) presented to Congress its proposal for an industry-controlled rating system; in September of that same year, the Entertainment Software Rating Board (ESRB) was unveiled.'),
 (0.014081788072994273,
  'Video game content rating systems can be used as the basis for laws that cover the sales of video games to minors, such as in Australia.'),
 (0.01383469

In [93]:
a1 = 'Silicon Valley has looked into the future and seen what many of us are seeing – automation and technology are replacing jobs faster than businesses are creating them. What started as a mere murmurings has reverberated louder into an open call from some of the wealthiest businesspeople in the United States including Mark Zuckerberg and Elon Musk for universal basic income.Universal basic income is a stipend paid by the government to individuals simply for being alive. The idea began in Europe and has been gaining momentum in several countries. The principal behind the idea is that it serves as a safety net for unemployment due to market forces such as automation. Several countries have begun experimenting with a basic income including an experiment run in Ontario, Canada, which distributed a stipend of $12, 616 as its basic income. The idea has started to gain traction in America.Hawaii recently became the first state to make a major push towards a universal basic income. As of this May, the Hawaii state legislature adopted HCR 89 calling for the Department of Labor and Industrial Relations and the Department of Business, Economic Development, and Tourism to convene a basic income economic security working group to begin to structure a universal basic income. The relevant text from the resolution gives the group’s focus:WHEREAS, while the United States is the wealthiest nation in the world, many families, individuals, and businesses in Hawaii have been struggling to keep pace with the increasing cost of living as economic inequality widens the gap between a few top earners and the middle and lower class, the latter of which has seen its overall share of income decline in recent decades; and WHEREAS, efforts to increase wages, benefits, and working conditions are important steps to assist local families in the short-term, but a paradigm shift in policy will soon be necessary as automation, innovation, and disruption begin to rapidly worsen economic inequality by displacing significant numbers of jobs in Hawaii’s transportation, food service, tourism, retail, medical, legal, insurance, and other sectors …Hawaii state representative Chris Lee led the push towards a basic income and predicted that Hawaii’s future economic troubles due to Silicon Valley innovations like self-driving cars and Airbnb to be potentially hazardous to Hawaii’s tourist-dependent economy. Though this seems to be an anticipatory measure, however; as of April 2017, Hawaii’s unemployment rate was at a ten year low. This fact didn’t stop the Democrat dominated state legislature from approving the measure.'


In [95]:
a1 += "In other parts of the country, calls in for universal basic income are growing. In California, Silicon Valley has made well known their support for a basic income citing the necessity of social security from the jobs they will take away with their businesses. In the media, a cursory search of liberal publications reveals a bolder comfort with an idea of providing a stipend to people for doing nothing. In Georgia, former documentary assistant turned congressional candidate, Jon Ossoff, went above and beyond agreeing to a $15 minimum wage and instead gave a nod to his California benefactors by saying he would push for a “living wage” in the final debate for the Georgia Sixth Congressional district special election. It’s easy to see the sly rhetorical distinction between “living wage” and “basic income.”There is very little hope that this trend will change without forward-thinking leaders who will refuse to rest on lazy ideas like the universal basic income.Our politicians have been able to see technology replacing jobs for decades now and made little adjustments other than to make more individuals reliant on the government for their economic well being. I echo what I wrote in The Hill last September that automation would require lawmakers to be innovative in their approaches for creating jobs. Many lawmakers and those in the best position to see automation replacing jobs, such as CEO’s in Silicon Valley, it would appear are opting towards a more top-down income redistribution in the form of a universal basic income rather than wrestle with retraining workers, re-calibrating the economy, or encouraging innovation to offset job loss.This is to simply say: universal basic income is a lazy, socialist solution to future unemployment problems. There are dozens of issues with universal basic income, but I will list some of the bigger issues below. As to avoid being an armchair critic who only points out mistakes, I will also offer up some of my own solutions or alternatives to reposition our economy.First, universal basic income stifles economic growth through exorbitant tax rates. To pay for a basic income that would be a living income or sustainable income requires taxation of wage earners well above 50% for the highest and second highest tax brackets. Offsetting high cost of living cities like New York, San Francisco, and Washington, D.C. would necessitate a tax rate unsustainable for most Americans. This leads to either a subsidy for poor Americans, or force many low-income individuals to relocate out of major cities. Needless to say, this exodus hampers innovation.With our current immigration debate, this analysis presumes a comparable rate of illegal immigration and immigration as we have seen in recent years. If history is any indication, the moment a universal basic income comes into law for U.S. citizens, Mayor de Blasio and Governor Cuomo will rush to the front of public debate to argue that universal basic income should be extended to undocumented workers as well since it’s a human right. If that happens, keep your wallets open because that’s going to hurt.Second, universal basic income is a mere proxy for socialism. The plan relies on the assumption that the income will be a social safety net to hedge against unemployment. In a perfect world, individuals would be able to save to offset economic unpredictability. This is an interesting argument to hear coming from a Left who finds the thought of alternatives to social security and Obamacare in the form of private savings account abhorrent. If liberals don’t trust Americans to allocate their money correctly, how can they reasonably suspect that this universal income will be treated differently to realistically serve as a safety net?Finally, a basic universal income only hedges against future unemployment rather than creating opportunities in the present to anticipate economic changes. If automation continues to replace jobs, more Americans will shift from universal basic income being a subsidy to their earned wages towards it being a necessity in unemployment. It’s easy to see how this would spiral the economy into the garbage heap. As workers exit the workforce, fewer workers pay into the pool for distribution meaning that there is less money for the same amount of government handouts. To accommodate the gap, either taxes would have to go up on higher income earners to account for a depleting workforce, or businesses and the government would have to create alternatives to get people to work, or the government would have to require businesses to hire individuals. (Beginning to sound a bit like Atlas Shrugged.) All are bad alternatives. Creating a safety net that promises no ability to put people to work is giving them $1,000, not teaching them to build boats or sail, and asking them to get off an island.There is still time to right the ship. As I said at the outset, nay saying without a solution is as empty as the politicians that will inevitably thrust the universal basic income upon us. There are many avenues to avoiding the calamity from unemployment Silicon Valley predicts from their innovation. I will list several here.First, public schools and innovative STEM charter schools need the resources to teach STEM subjects and trade skills. Understanding how the workforce is being educated and what they are learning are far too often overlooked. For one, a realistic alternative would be to increase trade skills education during high school education while it is cheaper in order to provide career paths for students that do not seek a college degree while equipping them with a skill. It will be a long time before a plumber or electrician can be replaced by a machine. Access to STEM early in education will also improve female participation in STEM fields which has been noticeably absent and will lead to greater market participation in a field that will continue to grow as automation boxes out other fields. Targeting areas that lead to innovation and businesses can allocate resources will put people to work.Please do not misinterpret me to suggest that we need more education. I’d wager there are less than 1,000 people in America using an art history degree for its intended purpose. Instead, this is a proposal to reallocate resources to target types of education where humans will be valuable – trades that machines have difficulty replacing and STEM fields where humans create the machines who replace them.My second proposal is to unburden small businesses. Legislators should be in the business of making it easier for small businesses to operate and comply with government regulation. If state and federal regulation is reduced, small businesses will be best situated to create jobs cheaply. Healthcare constantly being in flux, taxation changes, and legal hurdles make running a small business and creating jobs difficult and in an economy that many predict will desperately need to create employment, government doesn’t need to be a formidable burden in that process.My proposals are not perfect. There are plenty of holes in these proposals and many require intensive policy debates. The debates are worth having because a universal basis income is a road towards a dying economy – settling on a universal basic income is both lazy and simplistic. Without real ingenuity in the way we approach our workforce, we may be doomed to an economy bereft of innovation and liberty."

In [106]:
textrank(b1)

[(0.027132983596283761,
  'Amazon is a big player in that small space, but Whole Foods is small within it, and there are plenty of other would-be players.Moreover, Amazon will tell a seemingly plausible story to regulators and courts—that it only means to buy a troubled firm in need of a boost and only to support its entry in a space that it could not successfully enter on its own.'),
 (0.025796669950298744,
  'But hopefully we also will see that a firm like Amazon embodies some things that are wrong with the policy and that it calls for action.That action could actually be taken in the Whole Foods case, and it is not even impossible that the Trump administration might take it.'),
 (0.023999653562320596,
  'And the significance of it could not be overstated, not only because of Amazon’s own size and danger but for the broader revitalization it would bring to this crucial policy.'),
 (0.022704851496836192,
  'The claim would be that this new acquisition and a series of prior acquisition